In [1]:
!pip install -qU \
  langchain-core \
  langchain-google-genai \
  langchain-community \
  scikit-image


In [2]:
import getpass
import os

if not os.environ.get("GOOGLE_API_KEY"):
  os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter API key for Google Gemini: ")

Enter API key for Google Gemini: ········


In [3]:
from langchain.chat_models import init_chat_model


llm = init_chat_model("gemini-2.0-flash", model_provider="google_genai", temperature=0.0) 

## Conversation Buffer Memory

In [4]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory(return_messages=True)

/tmp/ipykernel_9782/1448044083.py:3: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(return_messages=True)


In [5]:
memory.save_context(
    {"input": "Hi, my names in James"},
    {"output": "Hey James, what's up? I'm an AI model called Zeta."}
)
memory.save_context(
    {"input": "I am researching different types of conversational memory."},
    {"output": "That's interesting, what are some examples?"}
)
memory.save_context(
    {"input": "I am researching ConversationBufferMemory and ConversationBufferWindowMemory"},
    {"output": "That's interesting, what's thed difference"}
)
memory.save_context(
    {"input": "Buffer memory just store the enitre converation, right?"},
    {"output": "Thats makes sense, what about ConversationBufferWindowMemory"}
)
memory.save_context(
    {"input": "Buffer window memory stores last k messages, dropping the rest."},
    {"output": "Very cool!"}
)


In [6]:
memory.load_memory_variables({})

{'history': [HumanMessage(content='Hi, my names in James', additional_kwargs={}, response_metadata={}),
  AIMessage(content="Hey James, what's up? I'm an AI model called Zeta.", additional_kwargs={}, response_metadata={}),
  HumanMessage(content='I am researching different types of conversational memory.', additional_kwargs={}, response_metadata={}),
  AIMessage(content="That's interesting, what are some examples?", additional_kwargs={}, response_metadata={}),
  HumanMessage(content='I am researching ConversationBufferMemory and ConversationBufferWindowMemory', additional_kwargs={}, response_metadata={}),
  AIMessage(content="That's interesting, what's thed difference", additional_kwargs={}, response_metadata={}),
  HumanMessage(content='Buffer memory just store the enitre converation, right?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Thats makes sense, what about ConversationBufferWindowMemory', additional_kwargs={}, response_metadata={}),
  HumanMessage(conte

In [7]:
from langchain.chains import ConversationChain

chain = ConversationChain(
    llm = llm,
    memory = memory,
    verbose = True
)


/tmp/ipykernel_9782/2943370833.py:3: LangChainDeprecationWarning: The class `ConversationChain` was deprecated in LangChain 0.2.7 and will be removed in 1.0. Use :class:`~langchain_core.runnables.history.RunnableWithMessageHistory` instead.
  chain = ConversationChain(


In [8]:
chain.invoke({"input": "What is my name again?"})



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
[HumanMessage(content='Hi, my names in James', additional_kwargs={}, response_metadata={}), AIMessage(content="Hey James, what's up? I'm an AI model called Zeta.", additional_kwargs={}, response_metadata={}), HumanMessage(content='I am researching different types of conversational memory.', additional_kwargs={}, response_metadata={}), AIMessage(content="That's interesting, what are some examples?", additional_kwargs={}, response_metadata={}), HumanMessage(content='I am researching ConversationBufferMemory and ConversationBufferWindowMemory', additional_kwargs={}, response_metadata={}), AIMessage(content="That's interesting, what's thed difference", additional_kwargs={}

{'input': 'What is my name again?',
 'history': [HumanMessage(content='Hi, my names in James', additional_kwargs={}, response_metadata={}),
  AIMessage(content="Hey James, what's up? I'm an AI model called Zeta.", additional_kwargs={}, response_metadata={}),
  HumanMessage(content='I am researching different types of conversational memory.', additional_kwargs={}, response_metadata={}),
  AIMessage(content="That's interesting, what are some examples?", additional_kwargs={}, response_metadata={}),
  HumanMessage(content='I am researching ConversationBufferMemory and ConversationBufferWindowMemory', additional_kwargs={}, response_metadata={}),
  AIMessage(content="That's interesting, what's thed difference", additional_kwargs={}, response_metadata={}),
  HumanMessage(content='Buffer memory just store the enitre converation, right?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Thats makes sense, what about ConversationBufferWindowMemory', additional_kwargs={}, respons

## ConversationBufferMemory with RunnableWithMessageHistory

In [9]:
from langchain.prompts import (
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
    MessagesPlaceholder,
    ChatPromptTemplate
)

system_prompt = "You are a helpful assistant called Zeta."

prompt_template = ChatPromptTemplate.from_messages([
    SystemMessagePromptTemplate.from_template(system_prompt),
    MessagesPlaceholder(variable_name="history"),
    HumanMessagePromptTemplate.from_template("{query}")   
])


In [10]:
pipeline = (
    { 
        "query": lambda x: x["query"],
        "history": lambda x: x["history"]
    }
    | prompt_template
    | llm
)

In [11]:
from langchain_core.chat_history import InMemoryChatMessageHistory

chat_map = {}
def get_chat_history(session_id: str) -> InMemoryChatMessageHistory:
    if session_id not in chat_map:
        # if session ID doesn't exist, create a new chat history
        chat_map[session_id] = InMemoryChatMessageHistory()
    return chat_map[session_id]

In [12]:
from langchain_core.runnables.history import RunnableWithMessageHistory

pipeline_with_history = RunnableWithMessageHistory(
    pipeline,
    get_session_history=get_chat_history,
    input_messages_key="query",
    history_messages_key="history"
)

In [13]:
pipeline_with_history.invoke(
    {"query": "Hi, my name is James"},
    config={"session_id": "id_123"}
)

AIMessage(content="Hi James, it's nice to meet you! How can I help you today?", additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []}, id='run--af6717c4-0140-48c4-952d-32905b94331b-0', usage_metadata={'input_tokens': 14, 'output_tokens': 19, 'total_tokens': 33, 'input_token_details': {'cache_read': 0}})

In [14]:
pipeline_with_history.invoke(
    {"query": "What is my name again?"},
    config={"session_id": "id_123"}
)

AIMessage(content='Your name is James.', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []}, id='run--fbfcf1ee-5ceb-48e7-96af-9fa1060589df-0', usage_metadata={'input_tokens': 38, 'output_tokens': 6, 'total_tokens': 44, 'input_token_details': {'cache_read': 0}})

## ConversationBufferWindowMemory

In [15]:
from langchain.memory import ConversationBufferWindowMemory

memory = ConversationBufferWindowMemory(k=4, return_messages=True)

/tmp/ipykernel_9782/3216785012.py:3: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferWindowMemory(k=4, return_messages=True)


In [16]:
memory.save_context(
    {"input": "Hi, my names in James"},
    {"output": "Hey James, what's up? I'm an AI model called Zeta."}
)
memory.save_context(
    {"input": "I am researching different types of conversational memory."},
    {"output": "That's interesting, what are some examples?"}
)
memory.save_context(
    {"input": "I am researching ConversationBufferMemory and ConversationBufferWindowMemory"},
    {"output": "That's interesting, what's thed difference"}
)
memory.save_context(
    {"input": "Buffer memory just store the enitre converation, right?"},
    {"output": "Thats makes sense, what about ConversationBufferWindowMemory"}
)
memory.save_context(
    {"input": "Buffer window memory stores last k messages, dropping the rest."},
    {"output": "Very cool!"}
)

In [17]:
memory.load_memory_variables({})

{'history': [HumanMessage(content='I am researching different types of conversational memory.', additional_kwargs={}, response_metadata={}),
  AIMessage(content="That's interesting, what are some examples?", additional_kwargs={}, response_metadata={}),
  HumanMessage(content='I am researching ConversationBufferMemory and ConversationBufferWindowMemory', additional_kwargs={}, response_metadata={}),
  AIMessage(content="That's interesting, what's thed difference", additional_kwargs={}, response_metadata={}),
  HumanMessage(content='Buffer memory just store the enitre converation, right?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Thats makes sense, what about ConversationBufferWindowMemory', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='Buffer window memory stores last k messages, dropping the rest.', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Very cool!', additional_kwargs={}, response_metadata={})]}

In [18]:
chain = ConversationChain(
    llm = llm,
    memory = memory,
    verbose = True
)

In [19]:
chain.invoke({"input": "What is my name again"})



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
[HumanMessage(content='I am researching different types of conversational memory.', additional_kwargs={}, response_metadata={}), AIMessage(content="That's interesting, what are some examples?", additional_kwargs={}, response_metadata={}), HumanMessage(content='I am researching ConversationBufferMemory and ConversationBufferWindowMemory', additional_kwargs={}, response_metadata={}), AIMessage(content="That's interesting, what's thed difference", additional_kwargs={}, response_metadata={}), HumanMessage(content='Buffer memory just store the enitre converation, right?', additional_kwargs={}, response_metadata={}), AIMessage(content='Thats makes sense, what about Conversat

{'input': 'What is my name again',
 'history': [HumanMessage(content='I am researching different types of conversational memory.', additional_kwargs={}, response_metadata={}),
  AIMessage(content="That's interesting, what are some examples?", additional_kwargs={}, response_metadata={}),
  HumanMessage(content='I am researching ConversationBufferMemory and ConversationBufferWindowMemory', additional_kwargs={}, response_metadata={}),
  AIMessage(content="That's interesting, what's thed difference", additional_kwargs={}, response_metadata={}),
  HumanMessage(content='Buffer memory just store the enitre converation, right?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Thats makes sense, what about ConversationBufferWindowMemory', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='Buffer window memory stores last k messages, dropping the rest.', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Very cool!', additional_kwargs={}, respon

In [20]:
from pydantic import BaseModel, Field
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.messages import BaseMessage

class BufferWindowMessageHistory(BaseChatMessageHistory, BaseModel):
    messages: list[BaseMessage] = Field(default_factory=list)
    k: int = Field(default_factory=int)
    
    
    def __init__(self, k:int):
        super().__init__(k=k)
        print(f"Initializing BufferWindowMessageHistory with k={k}")
        
    
    def add_messages(self, messages: list[BaseMessage]) -> None:
        """
        Add messages to the history, removing any messages beyond
        the last `k` messages
        """
        
        self.messages.extend(messages)
        self.messages = self.messages[-self.k:]
        
        
    def clear(self) -> None:
        """
        clear the history
        """
        self.messages = []       
        
        

In [21]:
chat_map = {}
def get_chat_window_history(session_id: str, k: int = 4) -> BufferWindowMessageHistory:
    if session_id not in chat_map:
        chat_map[session_id] = BufferWindowMessageHistory(k=k)
    
    return chat_map[session_id]

In [22]:
from langchain_core.runnables import ConfigurableFieldSpec

pipeline_cwbm = (
    { 
        "query": lambda x: x["query"],
        "history": lambda x: x["history"]
    }
    | prompt_template
    | llm
)
 

pipeline_with_history_cwbm = RunnableWithMessageHistory(
    pipeline_cwbm,
    get_session_history = get_chat_window_history,
    input_messages_key = "query",
    history_messages_key = "history",
    history_factory_config = [
        ConfigurableFieldSpec(
            id = "session_id",
            annotation = str,
            name = "Session ID",
            description = "The session ID to use for the chat history",
            default = "id_default"
        ),
        ConfigurableFieldSpec(
            id = "k",
            annotation = int,
            name = "k",
            description = "The number of messages to keep in the history",
            default = 4
        )
    ]
    
)

In [24]:
pipeline_with_history_cwbm.invoke(
    {
        "query": "my name is James"  
    },
    config = {
        "configurable": {
            "session_id" : "id_k3",
            "k" : 3
        }
    }

)

AIMessage(content="You already told me that, James! Is there anything I can help you with? Perhaps you'd like to tell me something else about yourself, or ask me a question?", additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []}, id='run--aa8c285b-23d8-4d1c-b33e-3173baeaeb65-0', usage_metadata={'input_tokens': 30, 'output_tokens': 37, 'total_tokens': 67, 'input_token_details': {'cache_read': 0}})

In [25]:
pipeline_with_history_cwbm.invoke(
    {
        "query": "I am learning langchain"  
    },
    config = {
        "configurable": {
            "session_id" : "id_k3",
            "k" : 3
        }
    }

)

AIMessage(content="That's great, James! Langchain is a powerful tool. What are you hoping to achieve with Langchain? Knowing your goals will help me provide more relevant information and assistance.\n\nFor example, are you interested in:\n\n*   **Building a specific type of application?** (e.g., a chatbot, a document summarizer, a question-answering system)\n*   **Learning about a particular aspect of Langchain?** (e.g., chains, agents, memory, models)\n*   **Troubleshooting a problem you're encountering?**\n*   **Finding resources for learning Langchain?**\n\nTell me a bit more about what you're working on, and I'll do my best to help!", additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []}, id='run--b4f1f898-81bf-415f-bc9b-1e926c4ce5d2-0', usage_metadata={'input_tokens': 67, 'output_tokens': 155, 'total_tokens': 222, 'input_token_details': {'cache_read': 0

In [26]:
pipeline_with_history_cwbm.invoke(
    {
        "query": "My plan is to learn Langgraph and Google A2A, which one should I go"  
    },
    config = {
        "configurable": {
            "session_id" : "id_k3",
            "k" : 3
        }
    }

)

AIMessage(content="Okay, that's a great plan! Both LangGraph and Google's A2A (presumably you mean Vertex AI's Agent Builder, which is sometimes referred to as A2A) are powerful tools for building AI applications, but they serve different purposes and have different learning curves. Here's a breakdown to help you decide which to tackle first:\n\n**LangGraph:**\n\n*   **Focus:** Orchestration and state management for complex, multi-actor LLM applications. Think of it as a framework for building robust and reliable workflows involving multiple LLM calls, conditional logic, and state transitions.\n*   **Use Cases:** Ideal for applications that require complex decision-making, iterative refinement, and handling of different scenarios. Examples include:\n    *   Complex chatbots with multiple tools and conversational flows.\n    *   Automated agents that perform tasks involving multiple steps and dependencies.\n    *   Applications that require reasoning and planning.\n*   **Learning Curve:

In [34]:
chat_map["id_k3"].messages[2]

AIMessage(content="Okay, that's a great plan! Both LangGraph and Google's A2A (presumably you mean Vertex AI's Agent Builder, which is sometimes referred to as A2A) are powerful tools for building AI applications, but they serve different purposes and have different learning curves. Here's a breakdown to help you decide which to tackle first:\n\n**LangGraph:**\n\n*   **Focus:** Orchestration and state management for complex, multi-actor LLM applications. Think of it as a framework for building robust and reliable workflows involving multiple LLM calls, conditional logic, and state transitions.\n*   **Use Cases:** Ideal for applications that require complex decision-making, iterative refinement, and handling of different scenarios. Examples include:\n    *   Complex chatbots with multiple tools and conversational flows.\n    *   Automated agents that perform tasks involving multiple steps and dependencies.\n    *   Applications that require reasoning and planning.\n*   **Learning Curve:

[AIMessage(content='That makes sense, what about ConversationBufferWindowMemory?', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Buffer window memory stores the last k messages, dropping the rest.', additional_kwargs={}, response_metadata={}),
 AIMessage(content='Very cool!', additional_kwargs={}, response_metadata={})]